In [1]:
from llama_index.core.prompts import PromptTemplate

In [2]:
qa_prompt = """请仅根据提供的资源提供答案。
    当引用来自一个资源的信息时，
    请使用相应的编号来引用合适的资源。
    每个答案应至少包含一个资源引用。
    只有在你明确引用资源时，才需要引用来源。
    如果所有资源都无助于解答，你应该指出这一点。
    例如：
    资源1：
    傍晚时天空是红色的，早上时天空是蓝色的。
    资源2：
    当天空是红色时，水是湿润的。
    问题：水什么时候是湿润的？
    答案：水在天空是红色时是湿润的[2]，
    这种情况发生在傍晚[1]。
    现在轮到你了。以下是几个编号的信息资源：
    ------\n
    {context_str}
    ------\n
    问题：{query_str}
    答案："""


refine_prompt = """
    请仅根据提供的资源提供答案。
    当引用来自一个资源的信息时，
    请使用相应的编号来引用合适的资源。
    每个答案应至少包含一个资源引用。
    只有在你明确引用资源时，才需要引用来源。
    如果所有资源都无助于解答，你应该指出这一点。
    例如：
    资源1：
    傍晚时天空是红色的，早上时天空是蓝色的。
    资源2：
    当天空是红色时，水是湿润的。
    问题：水什么时候是湿润的？
    答案：水在天空是红色时是湿润的[2]，
    这种情况发生在傍晚[1]。
    现在轮到你了。
    我们已经提供了一个现成的答案：{existing_answer}
    以下是几个编号的信息资源。
    使用它们来完善现有的答案。
    如果提供的资源没有帮助，你将重复现有的答案。
    开始完善！
    ------\n
    {context_msg}
    ------\n
    问题：{query_str}
    答案：
"""


In [1]:

import logging

# Create a logger
logger = logging.getLogger("my_logger")
logger.setLevel(logging.DEBUG)  # Set the logging level to DEBUG

# Create handlers for writing to file and console
file_handler = logging.FileHandler("example.log")
console_handler = logging.StreamHandler()

# Set the logging level for handlers
file_handler.setLevel(logging.DEBUG)
console_handler.setLevel(logging.INFO)

# Create formatters and add it to handlers
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)



In [3]:
QA = PromptTemplate(qa_prompt)
REFINE = PromptTemplate(refine_prompt)

NameError: name 'PromptTemplate' is not defined

In [2]:
from load_from_chroma import get_index_from_chroma
from llama_index.llms.glm import GLM
from Embedding import BGEEbedding


## 初始化llm，嵌入模型以及retriever

In [8]:
llm_model = GLM(model="glm-4-flash",temperature = 0)
# llm_model = GLM(model="glm-4-flash",temperature = 0)

embedding_model = BGEEbedding()
law_db = get_index_from_chroma(
    path=".\\LawDb",collection="laws",
    embedding_model=embedding_model
)



In [5]:
from llama_index.core import PromptTemplate

def process_query(query):
    template = """
        你是一位专业的法律相关行业的咨询专家。当用户在对法律问题提问时，由于对法律相关知识的缺失，所以所提的问题会显得不是很专业。请结合用户所提出的问题进行意图识别后，根据其提问意图，对不专业的问题进行重写为专业的法律问题并输出。
        **注意**，请直接输出重组后的问题而不需要其他任何的东西。
            用户的问题：{law}
        """
    qa_template = PromptTemplate(template)
    resopnse = llm_model.predict(qa_template, law=query)

    word_template = PromptTemplate("""您是一位深耕法律领域的专业咨询顾问，您的职责在于针对所提出的法律疑问，精准识别提问核心，从中提炼出适宜检索相关法律条文的关键词组。这些词组需确保在chroma向量库中能有效匹配到相关法条，且每个词组均须与原始问题紧密相关。请将提炼的词组以中文形式输出，词组间使用英文逗号","分隔，格式为"XXXX的XXXXX"。请直接列出所有词组，无需其他附加内容。请确保至少提取一个词组，且对提取数量不设上限。请参考以下示例执行：

    示例：
    问题:故意杀人的刑事责任及具体刑期如何确定？
    输出:故意杀人的刑事责任,故意杀人的刑期确定
    问题:依据我国刑法，过失杀人行为的刑罚年限有何规定？
    输出:过失杀人行为的刑罚年限

    请依此范例进行输出。
        问题：{law_q}
    """)
    law_query = llm_model.predict(word_template, law_q=str(resopnse)).split(',')

    law_retriever = law_db.as_retriever()
    result = []
    for i in law_query:
        for j in law_retriever.retrieve(i):
            result.append(str(j))

    q_template = PromptTemplate("""
        你是一位深耕法律领域的专业咨询顾问。请从给出的法律法规中回答问题。
        法律条文：{law_s}
        问题：{law_q}
        请确保回答准确、专业，并基于给出的法律法规。同时，请确保回答符合法律专业术语的规范和标准。
        """)
    r = llm_model.predict(q_template, law_s=result, law_q=str(resopnse))
    return str(r)

In [9]:
law_searcher = law_db.as_query_engine(llm=llm_model)

In [15]:
query = "经济犯罪的量刑标准?"
response = process_query(query)
from IPython.display import Markdown, display
display(Markdown(f"""# 思维链结果:\n {response}"""))
display(Markdown(f"""# 直接使用引擎结果:\n{str(law_searcher.query(query))}"""))

# 思维链结果:
 经济犯罪的量刑标准确定，依据《中华人民共和国刑法》的相关规定，具体如下：

1. **根据犯罪事实、性质、情节和社会危害程度**：第六十一条规定，对于犯罪分子决定刑罚时，应当根据犯罪的事实、犯罪的性质、情节和对于社会的危害程度，依照本法的有关规定判处。

2. **从重、从轻、减轻处罚情节**：第六十二条规定，犯罪分子具有本法规定的从重处罚、从轻处罚情节的，应当在法定刑的限度以内判处刑罚。

3. **减轻处罚情节**：第六十三条规定，犯罪分子具有本法规定的减轻处罚情节的，应当在法定刑以下判处刑罚；如果本法规定有数个量刑幅度的，应当在法定量刑幅度的下一个量刑幅度内判处刑罚。犯罪分子虽然不具有本法规定的减轻处罚情节，但根据案件的特殊情况，经最高人民法院核准，也可以在法定刑以下判处刑罚。

4. **违法所得的追缴和没收**：第六十四条规定，犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库。

5. **刑罚的种类**：第三十二条规定，刑罚分为主刑和附加刑。主刑包括管制、拘役、有期徒刑、无期徒刑和死刑；附加刑包括罚金、剥夺政治权利和没收财产。

6. **赔偿经济损失**：第三十六条规定，由于犯罪行为而使被害人遭受经济损失的，对犯罪分子除依法给予刑事处罚外，并应根据情况判处赔偿经济损失。

综上所述，经济犯罪的量刑标准是综合考量犯罪的事实、性质、情节、社会危害程度以及犯罪分子的具体情节，依法在法定刑的范围内进行判处，并可能涉及追缴违法所得、赔偿经济损失等措施。

# 直接使用引擎结果:
经济犯罪的量刑标准根据犯罪的事实、性质、情节和对于社会的危害程度，依照相关法律规定判处。具体而言，对于具有从重、从轻或减轻处罚情节的犯罪分子，应在法定刑的限度内判处刑罚。例如，抢劫公私财物的，根据抢劫的具体情形，可能被判处三年以上十年以下有期徒刑，并处罚金；若情节严重，如入户抢劫、抢劫银行等，则可能被判处十年以上有期徒刑、无期徒刑或者死刑，并处罚金或者没收财产。盗窃公私财物的，根据盗窃的数额和情节，可能被判处三年以下有期徒刑、拘役或者管制，并处罚金；若情节严重，则可能被判处三年以上十年以下有期徒刑，并处罚金。

In [15]:
from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore


class RetrieverEvent(Event):
    """Result of running retrieval"""
    nodes: list[NodeWithScore]

class CreateCitationsEvent(Event):
    """Add citations to the nodes."""

    nodes: list[NodeWithScore]

In [16]:
from llama_index.core import VectorStoreIndex
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)

from llama_index.core.schema import (
    MetadataMode,
    NodeWithScore,
    TextNode,
)

from llama_index.core.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

from typing import Union, List
from llama_index.core.node_parser import SentenceSplitter


class CitationQueryEngineWorkflow(Workflow):
    @step
    async def retrieve(
            self, ctx: Context, ev: StartEvent
    ) -> Union[RetrieverEvent, None]:
        """Entry point for RAG, triggered by a StartEvent with `query`."""
        query = ev.get("query")
        if not query:
            return None

        print(f"Query the database with: {query}")

        # store the query in the global context
        await ctx.set("query", query)

        if ev.index is None:
            print("Index is empty, load some documents before querying!")
            return None
        indexes: VectorStoreIndex = ev.index
        retriever = indexes.as_retriever(similarity_top_k=5)
        nodes = retriever.retrieve(query)
        print(f"Retrieved {len(nodes)} nodes.")
        return RetrieverEvent(nodes=nodes)

    @step
    async def create_citation_nodes(self, ev: RetrieverEvent) -> CreateCitationsEvent:
        """
        Modify retrieved nodes to create granular sources for citations.

        Takes a list of NodeWithScore objects and splits their content
        into smaller chunks, creating new NodeWithScore objects for each chunk.
        Each new node is labeled as a numbered source, allowing for more precise
        citation in query results.

        Args:
            nodes (List[NodeWithScore]): A list of NodeWithScore objects to be processed.

        Returns:
            List[NodeWithScore]: A new list of NodeWithScore objects, where each object
            represents a smaller chunk of the original nodes, labeled as a source.
        """
        nodes = ev.nodes

        new_nodes: List[NodeWithScore] = []

        text_splitter = SentenceSplitter(
            chunk_size=512,
            chunk_overlap=30, )

        for node in nodes:
            text_chunks = text_splitter.split_text(
                node.node.get_content(metadata_mode=MetadataMode.NONE)
            )

            for text_chunk in text_chunks:
                text = f"Source {len(new_nodes) + 1}:\n{text_chunk}\n"

                new_node = NodeWithScore(
                    node=TextNode.parse_obj(node.node), score=node.score
                )
                new_node.node.text = text
                new_nodes.append(new_node)
        return CreateCitationsEvent(nodes=new_nodes)

    @step
    async def synthesize(self, ctx: Context, ev: CreateCitationsEvent) -> StopEvent:
        """Return a streaming response using the retrieved nodes."""
        llm = GLM(is_function_calling_model=True)
        query = await ctx.get("query", default=None)

        synthesizer = get_response_synthesizer(
            llm=llm,
            text_qa_template=QA,
            refine_template=REFINE,
            response_mode=ResponseMode.COMPACT,
            use_async=True,
        )

        response = await synthesizer.asynthesize(query, nodes=ev.nodes)
        return StopEvent(result=response)

In [17]:
from load_from_chroma import get_index_from_chroma

index = get_index_from_chroma(path=""".\\LawDb""",collection="laws")

In [18]:
# Run a query
w = CitationQueryEngineWorkflow(timeout=1000)
result = await w.run(query="买卖毒品如何判刑", index=index)

Query the database with: 买卖毒品如何判刑
Retrieved 2 nodes.


In [19]:
from IPython.display import Markdown, display

display(Markdown(f"{result}"))
for res in result.source_nodes:
    print(res.node.get_text())

买卖毒品将根据毒品的种类和数量以及犯罪的具体情节来判刑[1][2]。如果走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的，将处十五年有期徒刑、无期徒刑或者死刑，并处没收财产[1]。如果走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，将处七年以上有期徒刑，并处罚金[1]。如果走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，将处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金[2]。

Source 1:
第七节 走私、贩卖、运输、制造毒品罪

第三百四十七条 走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。

走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：

（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；

（二）走私、贩卖、运输、制造毒品集团的首要分子；

（三）武装掩护走私、贩卖、运输、制造毒品的；

（四）以暴力抗拒检查、拘留、逮捕，情节严重的；

（五）参与有组织的国际贩毒活动的。

走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。

Source 2:
走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。

单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。

利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。

对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。

第三百四十八条 非法持有鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的，处七年以上有期徒刑或者无期徒刑，并处罚金；非法持有鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。



In [20]:
result.source_node

AttributeError: 'Response' object has no attribute 'source_node'

In [21]:
for res in result.source_nodes:
    print(res.node.get_text())

Source 1:
第七节 走私、贩卖、运输、制造毒品罪

第三百四十七条 走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。

走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：

（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；

（二）走私、贩卖、运输、制造毒品集团的首要分子；

（三）武装掩护走私、贩卖、运输、制造毒品的；

（四）以暴力抗拒检查、拘留、逮捕，情节严重的；

（五）参与有组织的国际贩毒活动的。

走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。

Source 2:
走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。

单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。

利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。

对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。

第三百四十八条 非法持有鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的，处七年以上有期徒刑或者无期徒刑，并处罚金；非法持有鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。



In [22]:
print(result.source_nodes)

[NodeWithScore(node=TextNode(id_='2bd8108a-1415-4e8e-9392-601c002fdf90', embedding=None, metadata={'file_path': 'd:\\code\\law_llama_system\\Law-Book\\7-刑法\\刑法.md', 'file_name': '刑法.md', 'file_size': 217091, 'creation_date': '2024-09-13', 'last_modified_date': '2024-01-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5453df9e-e883-4d10-aaae-fa52c18d2191', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'd:\\code\\law_llama_system\\Law-Book\\7-刑法\\刑法.md', 'file_name': '刑法.md', 'file_size': 217091, 'creation_date': '2024-09-13', 'last_modified_date': '2024-01-22'}, hash='a8d880f037c4caeb4f198d4a10875cb92eac8c49a3c93b06444c4fc365043fa1'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id=